In [10]:
!pip install --upgrade diffusers accelerate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

repo_id = "runwayml/stable-diffusion-v1-5"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [12]:
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

In [ ]:
# load the model
model = VGG16()
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summary
print(model.summary())

553467096/553467096 [==============================] - 14s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

get preview of dataset

In [15]:
import requests
# get dataset previeo of LAION aestestichs v5+ to get some prompts to generate, I do not need
# the entire dataset since the experiments want to be conducted with small samples of data

API_URL = "https://datasets-server.huggingface.co/first-rows?dataset=ChristophSchuhmann%2Fimproved_aesthetics_5plus&config=ChristophSchuhmann--improved_aesthetics_5plus&split=train"
def query():
    response = requests.request("GET", API_URL)
    return response.json()
data = query()

In [16]:
import pandas as pd

l = []
for i in range(len(data['rows'])):
  l.append(data['rows'][i]['row'])

data = pd.DataFrame(l)

In [17]:
data

,URL,TEXT,WIDTH,HEIGHT,similarity,punsafe,pwatermark,AESTHETIC_SCORE,hash,__index_level_0__
0,https://cdn.onekindesign.com/wp-content/upload...,Small Home Office Design Ideas: 28 Dreamy Home...,1188.0,1392.0,0.323254,0.000393,0.052337,5.852898,-6796036020840233336,3
1,https://cdn1.bigcommerce.com/server3400/9la52e...,Martine Dining Chair - Brown Leatherette,193.0,290.0,0.335804,0.000005,0.049582,5.071020,-3184521389948441742,7
2,https://img4640.weyesimg.com/uploads/fsszled-h...,waterproof light housing 3W 6W garden underwat...,800.0,800.0,0.314541,0.000026,0.125070,5.081889,2323462499383195015,8
3,https://cdn.shopify.com/s/files/1/2659/1590/pr...,Grimm's Wooden Toys Decorative Wooden Figures,875.0,700.0,0.301402,0.000325,0.400463,5.356396,1833435635425320828,21
4,https://trishcoonrod.com/wp-content/uploads/20...,"""""""2 Pears on Blue9"""""""" x 12""""""""""""""",2000.0,1200.0,0.298793,0.000370,0.042921,5.799916,2691700427382941650,24
...,...,...,...,...,...,...,...,...,...,...
95,https://www.sortrature.com/wp-content/uploads/...,Colorful Hair From Orange to Purple,500.0,750.0,0.303223,0.002405,0.425775,5.343545,-7717953855188095825,330
96,https://mexicansilver.co.uk/wp-content/uploads...,Silver with onyx droplet necklace,300.0,300.0,0.341225,0.000148,0.119328,5.291470,5322648296936439789,331
97,https://static.wixstatic.com/media/87a8f35c74b...,Aircraft Maintenance,490.0,368.0,0.310847,0.014155,0.091655,5.510855,7534732215896500262,332
98,http://images.tastespotting.com/uploads/thumbn...,Seafood Gumbo: A glorious gumbo is southern co...,250.0,250.0,0.317196,0.000007,0.040687,5.617904,-8372928656198747250,333


Download original imagews from dataset preview

In [ ]:
from google.colab import files as FILE
import os

for i in range(len(data)):
  try:
    img_data = requests.get(data.iloc[i,0]).content
    with open(data.iloc[i,1]+".jpg", 'wb') as handler:
        handler.write(img_data)
  except:
    pass

#FILE.download('image_name.jpg')
#os.remove('image_name.jpg') # to save up space

In [ ]:
import shutil
for i in range(len(data)):
  try:
    shutil.move(data.iloc[i,1]+".jpg" , "/content/dataset") 
  except:
    pass

In [ ]:
shutil.make_archive("/content/dataset", 'zip', "/content/dataset")

NameError: ignored

from image to text

In [25]:
from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [37]:
data.iloc[2,0]

'https://img4640.weyesimg.com/uploads/fsszled-housing.com/images/15663791281841.jpg?imageView2/2/w/800/h/800/q/75'

GENERATE

In [27]:
def generations(prompt, directory, steps=500, num_inference_steps = 20):
  for i in range(steps):
    image = pipe(prompt, num_inference_steps=num_inference_steps).images[0]
    image.save(directory +'/'+str(i) + prompt +".png")

modify parameters:

In [51]:
steps = 50
num_inference_steps = 20
prompt = 'two men in space suits sitting at a desk, a jigsaw puzzle by Jeff A. Menges, reddit, space art, associated press photo, sense of awe, redshift'
directory = '/content/6'

In [49]:
data.iloc[6,0]

'https://images01.military.com/sites/default/files/styles/full/public/media/news/people/2014/05/052814-mark-and-scott-kelly.jpg?itok=IIjSKi7g'

In [52]:
generations(prompt, directory, steps = steps, num_inference_steps = num_inference_steps)

#create zip file:
import shutil

shutil.make_archive(directory, 'zip', directory)

#from google.colab import files
#files.download(directory + '.zip')

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

'/content/6.zip'

extract features from images

In [ ]:
def features(directory, prompt, steps = 500):

  features = {}
  for i in range(steps):
      image_path = directory+'/' + str(i)+ prompt +  ".png"
      
      # load the image
      img = load_img(image_path, target_size=(224, 224))
      # convert pixel to numpy array
      img = img_to_array(img)
      # reshape the image for the model
      img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
      # preprocess the image
      img = preprocess_input(img)
      # extract features
      feature = model.predict(img, verbose=0)
      # store feature
      features[image_path] = feature
    
  return features

In [ ]:
f = features(directory, prompt, steps = steps)

originale, RICORDARSI DI CARICARE ORIGINAL E IMPOSTARE PATH

In [ ]:
#ORIGINALE
image_path = '/content/Home-Office-Library-Ideas-11-1-Kindesign original.jpg'
  # load the image
img = load_img(image_path, target_size=(224, 224))
    # convert pixel to numpy array
img = img_to_array(img)
    # reshape the image for the model
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    # preprocess the image
img = preprocess_input(img)
    # extract features
original_feature = model.predict(img, verbose=0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = []
for i in range(steps):
  try:
    v1 = original_feature[0]
    v2 = f[directory +'/'+ str(i)+ prompt +  ".png"][0]
    similarity = cosine_similarity(v1.reshape(1, -1), v2.reshape(1, -1))
    similarities.append((similarity[0][0], i))
  except:
    pass


similarities.sort(reverse = True)
print("top 10 Cosine Similarity:", similarities[:10]) #il background mi fotte, soggetto sbagliato per il test

top 10 Cosine Similarity: [(0.7667277, 286), (0.74977225, 249), (0.74625576, 217), (0.7446259, 473), (0.74098164, 371), (0.7379562, 379), (0.7376402, 0), (0.7375316, 295), (0.7361331, 327), (0.7349434, 223)]


In [ ]:
with open('/content/drive/My Drive/dati_attacchi.txt', 'a') as f: 
    f.write('\n\n' + prompt +'\n'+'STEPS: '+ str(num_inference_steps) +'\n' + str(similarities[:10]))

In [ ]:
#756, 47, 112, 132, 986
'''
seguire paper:
Carlini_2023_Extracting_Training_Data_from_Diffusion_Models.pdf

. Instead, we first embed each image
to a 512 dimensional vector using CLIP [54], and then
perform the all-pairs comparison between images in this
lower-dimensional space (increasing efficiency by over
1500×). We count two examples as near-duplicates if
their CLIP embeddings have a high cosine similarity. For
each of these near-duplicated images, we use the corresponding captions as the input to our extraction attack

poi devo generare tipo 500 immagine per ogni prompt delle immagini che sono tra loro più simili.

e poidevo fare membership inference sulle immagini generate per vedere se fanno parte o no del training set, se fanno parte vuol dire che ho estratto
le immagini.

potrei fare white box se ho la loss function se non ce l'ho devo seguire la paper.

trova immagini probabilmente simili con un clique finding su 500 immagini generate, una clique è prob memorizzata se almeno 10 immagini nel grafo delle 500 immagini
sono near identical. 

(potrei trovare un metodo più semplice io, oppure provare alla brutta con loss function e trovare un metodo per
tarare la threshold sotto la quale un elemento è considerato presente nel training set, cioè al di sotto di un certo valore della loss function.)
poi dovrei fare qualcosa di simile su un modello black box e le immagini che estraggo VERFICARE SE ABBIANO PROBLEMI DI COPYRIGHT O DI PRIVACY

come verificare che abbiano problemi di copyright o privacy?? magari chiedere al prof.
'''